In [1]:
from tensorflow import keras
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255.0,
                                                       vertical_flip = True,
                                                       horizontal_flip = True)

In [2]:
from keras.applications import ResNet50

conv_base = ResNet50(weights = "imagenet",
include_top = False,
input_shape = (224, 224, 3))
conv_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [3]:
import splitfolders
inputfolder = 'Animals/raw-img'
output = 'Animals'
splitfolders.ratio(inputfolder, output = output, seed = 40, ratio = (0.8, 0.1, 0.1))

Copying files: 26179 files [17:06, 25.51 files/s]


In [6]:
trainGen = datagen.flow_from_directory(
    r'C:\Users\vidha\Machine Learning\Animals\train',
    target_size = (224,224),
    batch_size = 32,
    shuffle = True,
    class_mode = 'categorical'
)

testGen = datagen.flow_from_directory(
    r'C:\Users\vidha\Machine Learning\Animals\test',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)

valGen = datagen.flow_from_directory(
    r'C:\Users\vidha\Machine Learning\Animals\val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 20938 images belonging to 10 classes.
Found 2627 images belonging to 10 classes.
Found 2614 images belonging to 10 classes.


In [8]:
import numpy as np
def extractfeatures(sample_count, generator):
    i = 0
    batch_size = 32
    features = np.zeros(shape=(sample_count, 7, 7, 2048))
    labels = np.zeros(shape=(sample_count, 10))
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i+1) * batch_size] = features_batch
        labels[i * batch_size: (i+1) * batch_size] = labels_batch
        i += 1
        if ((i * batch_size % 1000) == 0 ):
            print("processed size =", i * batch_size)
        if i * batch_size >= sample_count:
            break
    return features, labels

In [9]:
train_features, train_labels = extractfeatures(20938, trainGen)
test_features, test_labels = extractfeatures(2627, testGen)
val_features, val_labels = extractfeatures(2614, valGen)

1/1 [==============================] - 6s 6s/step
processed size = 4000
1/1 [==============================] - 5s 5s/step
processed size = 8000
1/1 [==============================] - 5s 5s/step
processed size = 12000
1/1 [==============================] - 5s 5s/step
processed size = 16000
1/1 [==============================] - 5s 5s/step
processed size = 20000
1/1 [==============================] - 3s 3s/step


In [10]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [11]:
from imutils import paths
totalTrain = len(list(paths.list_images(r'C:\Users\vidha\Machine Learning\Animals\train')))
totalVal = len(list(paths.list_images(r'C:\Users\vidha\Machine Learning\Animals\val')))
totalTest = len(list(paths.list_images(r'C:\Users\vidha\Machine Learning\Animals\test')))

In [12]:
steptrain = totalTrain // 32
stepval = totalVal // 32

In [15]:
model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

In [16]:
model.fit(train_features, train_labels, steps_per_epoch = steptrain, validation_data = (val_features, val_labels), validation_steps = stepval, epochs = 50)

MemoryError: Unable to allocate 7.83 GiB for an array with shape (20938, 7, 7, 2048) and data type float32